In [62]:
import torch
import torch.nn.functional as F

words = open("names.txt").read().splitlines()

letters = sorted(list(set(''.join(words)))) #all letters from words

stoi = {l:i+1 for i, l in enumerate(letters)}
stoi['.'] = 0 #add '.' as start or end of the word
itos = {i:l for l, i in stoi.items()}
num_letters = len(stoi)

xs, ys = [], []

for word in words:
  chars = ['.'] + list(word) + ['.']
  for ch1, ch2 in zip(chars, chars[1:]):
      xs.append(stoi[ch1])
      ys.append(stoi[ch2])
      
xs = torch.tensor(xs)
ys = torch.tensor(ys)

xenc = F.one_hot(xs, num_classes=num_letters).float()
print(xenc)
xenc.shape

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.]])


torch.Size([228146, 27])

In [42]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((num_letters, num_letters), generator=g, requires_grad=True)

In [72]:
# gradient descent, training
learning_rate = 50.0
probs = []
epoch = 20
num = xs.nelement()

for _ in range(epoch):
  # forward pass
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  
  loss = -probs[torch.arange(num), ys].log().mean() #+ 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data -= learning_rate * W.grad

2.466810464859009
2.466702461242676
2.4665966033935547


In [78]:
def generateName(g):
    result = ''
    ind = 0

    while True:
        xenc = F.one_hot(torch.tensor([ind]), num_classes=num_letters).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        probs = counts / counts.sum(1, keepdims=True) # probabilities for next character

        ind = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        if ind == 0: break
        result += itos[ind]

    return result

g = torch.Generator().manual_seed(21474836327)
for _ in range(5):
    print(generateName(g))

klan
cin
ynaki
ldalmorsama
are
